# Molstar Widget for Molecular Visualization
This notebook demonstrates how to create a widget that embeds the Molstar molecular viewer in Jupyter for interactive molecular visualization.

In [107]:
import os
import subprocess
import time
import shutil
import tempfile
import uuid
import sys
import ipywidgets as widgets
from ipyfilechooser import FileChooser

In [108]:
class MultiCORSFileServer:
    """
    HTTP server with CORS support that can serve multiple files
    using an external server script
    """
    def __init__(self, port=8765):
        self.port = port
        self.server_process = None
        self.temp_dir = tempfile.mkdtemp(prefix="molstar_files_")
        self.served_files = {}  # Dictionary to track all served files
        
    def start(self):
        """Start the file server if not running"""
        if self.server_process is not None:
            return
            
        # Path to our external server script
        script_path = os.path.join(os.getcwd(), "cors_skript.py")
        
        # Check if the script exists
        if not os.path.exists(script_path):
            print(f"Error: Server script not found at {script_path}")
            print("Please create the cors_http_server.py file with the provided code.")
            return
            
        # Start the server process using the external script
        self.server_process = subprocess.Popen(
            [sys.executable, script_path, str(self.port), self.temp_dir],
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE
        )
        
        # Give it a moment to start
        time.sleep(1)
        print(f"CORS-enabled file server started on port {self.port}")
        
    def stop(self):
        """Stop the file server and clean up"""
        if self.server_process is not None:
            self.server_process.terminate()
            self.server_process = None
            # Clean up temp directory
            shutil.rmtree(self.temp_dir, ignore_errors=True)
            self.served_files = {}  # Clear the file tracking
            print("File server stopped")
            
    def __del__(self):
        """Ensure server is stopped when object is deleted"""
        self.stop()
        
    def serve_file(self, filepath):
        """
        Serve a specific file and return its URL.
        If the file is already served, return its existing URL.
        
        Parameters:
        -----------
        filepath : str
            Path to the file on the server
            
        Returns:
        --------
        str
            URL to access the file
        """
        # Check if this file is already being served
        if filepath in self.served_files:
            return self.served_files[filepath]
                
        # Ensure server is running
        self.start()
        
        # Create a unique filename to avoid collisions
        unique_prefix = uuid.uuid4().hex[:8]
        timestamp = int(time.time())
        original_filename = os.path.basename(filepath)
        filename = f"{original_filename}"
        dest_path = os.path.join(self.temp_dir, filename)
        
        # Copy the file
        shutil.copy2(filepath, dest_path)
        
        # Store the URL and return it
        file_url = f"http://localhost:{self.port}/{filename}"
        self.served_files[filepath] = file_url
        
        return file_url
        
    def remove_file(self, filepath):
        """
        Stop serving a specific file
        
        Parameters:
        -----------
        filepath : str
            Path to the original file that was served
        """
        if filepath in self.served_files:
            # Get the filename from the URL
            url = self.served_files[filepath]
            filename = url.split('/')[-1]
            
            # Remove the file from the temp directory
            dest_path = os.path.join(self.temp_dir, filename)
            if os.path.exists(dest_path):
                os.remove(dest_path)
                
            # Remove from tracking dict
            del self.served_files[filepath]
    
    def list_served_files(self):
        """
        Return a list of currently served files
        
        Returns:
        --------
        list
            List of (filepath, url) tuples for all currently served files
        """
        return [(path, url) for path, url in self.served_files.items()]

In [118]:
def create_molstar_with_server_files(height=600, width=800):
    """
    Create a Molstar viewer with server-side file selection
    
    Parameters:
    -----------
    height : int
        Height of the viewer in pixels
    width : int
        Width of the viewer in pixels
    """
    
    # Create a file server instance
    file_server = MultiCORSFileServer(9998)
    
    # Create a unique ID for the iframe
    iframe_id = f"molstar-frame-{uuid.uuid4().hex}"
    
    # Create the iframe for Molstar
    iframe = widgets.HTML(
        value=f'<iframe id="{iframe_id}" src="https://molstar.org/viewer/" width="{width}" height="{height}" frameBorder="0"></iframe>'
    )
    
    # Create file chooser widget
    start_dir = "/home/jovyan" if os.path.exists("/home/jovyan") else os.getcwd()
    file_chooser = FileChooser(start_dir)
    
    # File extension filters - using proper format for FileChooser
    # For ipyfilechooser filters, use a list of glob patterns like *.pdb
    structure_extensions = ['*.pdb', '*.cif', '*.mmcif', '*.sdf', '*.mol2']
    trajectory_extensions = ['*.xtc', '*.trr', '*.dcd', '*.gro']
    volumetric_extensions = ['*.dx', '*.cube', '*.mrc', '*.ccp4']
    
    # File type selector
    file_type = widgets.Dropdown(
        options=[
            ('Structure files', 'structure'),
            ('Trajectory files', 'trajectory'),
            ('Volumetric data', 'volumetric'),
            ('All files', 'all')
        ],
        value='structure',
        description='File type:',
        style={'description_width': 'initial'}
    )
    
    # Update file filter based on selection
    def update_filter(change):
        current_dir = file_chooser.selected_path or file_chooser.default_path
        
        if change['new'] == 'structure':
            file_chooser.filter_pattern = structure_extensions
        elif change['new'] == 'trajectory':
            file_chooser.filter_pattern = trajectory_extensions
        elif change['new'] == 'volumetric':
            file_chooser.filter_pattern = volumetric_extensions
        else:
            file_chooser.filter_pattern = ['*']
        
        # Force refresh by resetting the widget
        file_chooser.reset(path=current_dir)
        
        # Debug output
        print(f"Filter set to: {file_chooser.filter_pattern}")
    
    # Set initial filter
    file_chooser.filter_pattern = structure_extensions
    
    # Observe changes to the dropdown
    file_type.observe(update_filter, names='value')
    
    # Set initial filter
    file_chooser.filter_pattern = structure_extensions
    
    # Status message
    status = widgets.HTML(value="")
    
    # Currently served files list
    served_files = widgets.Select(
        options=[],
        description='Served files:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '100%', 'height': '100px'}
    )
    
    # Function to update the served files list
    def update_served_files_list():
        files = file_server.list_served_files()
        served_files.options = [(os.path.basename(path), path) for path, url in files]
    
    # Load button
    load_btn = widgets.Button(
        description='Load File',
        button_style='primary',
        icon='upload'
    )
    
    # Function to load the selected file
    def on_load_clicked(b):
        if not file_chooser.selected:
            status.value = '<span style="color: red">Please select a file first</span>'
            return
            
        try:
            # Serve the file via our server
            file_path = file_chooser.selected
            file_url = file_server.serve_file(file_path)
            
            # Update the Molstar iframe to load this file
            js_load_code = f"""
            <script>
                (function() {{
                    document.getElementById('{iframe_id}').src = 
                        'https://molstar.org/viewer/index.html?url={file_url}';
                }})();
            </script>
            """
            
            status.value = (
                f'<span style="color: green">Loaded: {os.path.basename(file_path)}</span>'
                f'{js_load_code}'
            )
            
            # Update the served files list
            update_served_files_list()
            
        except Exception as e:
            status.value = f'<span style="color: red">Error: {str(e)}</span>'
    
    load_btn.on_click(on_load_clicked)
    
    # Remove file button
    remove_file_btn = widgets.Button(
        description='Remove Selected',
        button_style='warning',
        icon='trash',
        disabled=False
    )
    
    # Function to remove a file from the server
    def on_remove_file(b):
        if not served_files.value:
            status.value = '<span style="color: orange">Please select a file to remove</span>'
            return
            
        try:
            file_path = served_files.value
            file_server.remove_file(file_path)
            status.value = f'<span style="color: blue">Removed: {os.path.basename(file_path)}</span>'
            
            # Update the served files list
            update_served_files_list()
            
        except Exception as e:
            status.value = f'<span style="color: red">Error removing file: {str(e)}</span>'
    
    remove_file_btn.on_click(on_remove_file)
    
    # Load from served files button
    load_served_btn = widgets.Button(
        description='Load Selected',
        button_style='info',
        icon='refresh',
        disabled=False
    )
    
    # Function to load a file that's already being served
    def on_load_served(b):
        if not served_files.value:
            status.value = '<span style="color: orange">Please select a file to load</span>'
            return
            
        try:
            file_path = served_files.value
            file_url = file_server.served_files[file_path]
            
            # Update the Molstar iframe to load this file
            js_load_code = f"""
            <script>
                (function() {{
                    document.getElementById('{iframe_id}').src = 
                        'https://molstar.org/viewer/index.html?url={file_url}';
                }})();
            </script>
            """
            
            status.value = (
                f'<span style="color: green">Loaded: {os.path.basename(file_path)}</span>'
                f'{js_load_code}'
            )
            
        except Exception as e:
            status.value = f'<span style="color: red">Error loading file: {str(e)}</span>'
    
    load_served_btn.on_click(on_load_served)
    
    # Create UI layout
    file_selector = widgets.VBox([
        widgets.HTML(value='<h3>Select file from server</h3>'),
        file_type, 
        file_chooser,
        widgets.HBox([load_btn]),
        widgets.HTML(value='<h3>Currently served files</h3>'),
        served_files,
        widgets.HBox([load_served_btn, remove_file_btn]),
        status
    ])
    
    # Add a cleanup button
    cleanup_btn = widgets.Button(
        description='Stop File Server',
        button_style='danger',
        icon='trash'
    )
    
    def on_cleanup(b):
        file_server.stop()
        status.value = '<span style="color: blue">File server stopped</span>'
        update_served_files_list()
        
    cleanup_btn.on_click(on_cleanup)
    
    # Assemble the final widget
    tabs = widgets.Tab([file_selector])
    tabs.set_title(0, "Server Files")
    
    # Complete widget with cleanup option
    main_widget = widgets.VBox([
        iframe,
        tabs,
        cleanup_btn
    ])
    
    return main_widget

In [120]:
molstar_viewer = create_molstar_with_server_files(height=600, width=1200)
display(molstar_viewer)

In [34]:
class MultiFileServer:
    """
    Enhanced HTTP server to expose multiple files to Molstar
    """
    def __init__(self, port=8765):
        self.port = port
        self.server_process = None
        self.temp_dir = tempfile.mkdtemp(prefix="molstar_files_")
        self.served_files = {}  # Dictionary to track all served files
        
    def start(self):
        """Start the file server if not running"""
        if self.server_process is not None:
            return
            
        # Use Python's built-in HTTP server
        cmd = ["python", "-m", "http.server", str(self.port), "--directory", self.temp_dir]
        self.server_process = subprocess.Popen(
            cmd, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.PIPE
        )
        
        # Give it a moment to start
        time.sleep(1)
        print(f"File server started on port {self.port}")
        
    def stop(self):
        """Stop the file server and clean up"""
        if self.server_process is not None:
            self.server_process.terminate()
            self.server_process = None
            # Clean up temp directory
            shutil.rmtree(self.temp_dir, ignore_errors=True)
            self.served_files = {}  # Clear the file tracking
            print("File server stopped")
            
    def __del__(self):
        """Ensure server is stopped when object is deleted"""
        self.stop()
        
    def serve_file(self, filepath):
        """
        Serve a specific file and return its URL.
        If the file is already served, return its existing URL.
        
        Parameters:
        -----------
        filepath : str
            Path to the file on the server
            
        Returns:
        --------
        str
            URL to access the file
        """
        # Check if this file is already being served
        if filepath in self.served_files:
            return self.served_files[filepath]
                
        # Ensure server is running
        self.start()
        
        # Create a unique filename to avoid collisions
        unique_prefix = uuid.uuid4().hex[:8]
        original_filename = os.path.basename(filepath)
        filename = f"{original_filename}"
        dest_path = os.path.join(self.temp_dir, filename)
        
        # Copy the file
        shutil.copy2(filepath, dest_path)
        
        # Store the URL and return it
        file_url = f"http://localhost:{self.port}/{filename}"
        self.served_files[filepath] = file_url
        
        return file_url
        
    def remove_file(self, filepath):
        """
        Stop serving a specific file
        
        Parameters:
        -----------
        filepath : str
            Path to the original file that was served
        """
        if filepath in self.served_files:
            # Get the filename from the URL
            url = self.served_files[filepath]
            filename = url.split('/')[-1]
            
            # Remove the file from the temp directory
            dest_path = os.path.join(self.temp_dir, filename)
            if os.path.exists(dest_path):
                os.remove(dest_path)
                
            # Remove from tracking dict
            del self.served_files[filepath]
    
    def list_served_files(self):
        """
        Return a list of currently served files
        
        Returns:
        --------
        list
            List of (filepath, url) tuples for all currently served files
        """
        return [(path, url) for path, url in self.served_files.items()]

In [35]:
def create_molstar_with_server_files(height=600, width=800):
    """
    Create a Molstar viewer with server-side file selection
    
    Parameters:
    -----------
    height : int
        Height of the viewer in pixels
    width : int
        Width of the viewer in pixels
    """
    # Create a file server instance
    file_server = MultiFileServer(port=9999)
    
    # Create a unique ID for the iframe
    iframe_id = f"molstar-frame-{uuid.uuid4().hex}"
    
    # Create the iframe for Molstar
    iframe = widgets.HTML(
        value=f'<iframe id="{iframe_id}" src="https://molstar.org/viewer/" width="{width}" height="{height}" frameBorder="0"></iframe>'
    )
    
    # Create file chooser widget
    file_chooser = FileChooser('/home/jovyan')
    
    # File extension filters
    structure_extensions = ['.pdb', '.cif', '.mmcif', '.sdf', '.mol2']
    trajectory_extensions = ['.xtc', '.trr', '.dcd', '.gro']
    volumetric_extensions = ['.dx', '.cube', '.mrc', '.ccp4']
    
    # File type selector
    file_type = widgets.Dropdown(
        options=[
            ('Structure files', 'structure'),
            ('Trajectory files', 'trajectory'),
            ('Volumetric data', 'volumetric'),
            ('All files', 'all')
        ],
        value='structure',
        description='File type:',
        style={'description_width': 'initial'}
    )
    
    # Update file filter based on selection
    def update_filter(change):
        if change['new'] == 'structure':
            file_chooser.filter_pattern = structure_extensions
        elif change['new'] == 'trajectory':
            file_chooser.filter_pattern = trajectory_extensions
        elif change['new'] == 'volumetric':
            file_chooser.filter_pattern = volumetric_extensions
        else:
            file_chooser.filter_pattern = ['*']
    
    file_type.observe(update_filter, names='value')
    file_chooser.filter_pattern = structure_extensions  # Initial filter
    
    # Load button
    load_btn = widgets.Button(
        description='Load File',
        button_style='primary',
        icon='upload'
    )
    
    # Status message
    status = widgets.HTML(value="")
    
    # Currently served files list
    served_files = widgets.Select(
        options=[],
        description='Served files:',
        disabled=False,
        style={'description_width': 'initial'},
        layout={'width': '100%', 'height': '100px'}
    )
    
    # Function to update the served files list
    def update_served_files_list():
        files = file_server.list_served_files()
        served_files.options = [(os.path.basename(path), path) for path, url in files]
    
    # Function to load the selected file
    def on_load_clicked(b):
        if not file_chooser.selected:
            status.value = '<span style="color: red">Please select a file first</span>'
            return
            
        try:
            # Serve the file via our server
            file_path = file_chooser.selected
            file_url = file_server.serve_file(file_path)
            
            # Update the Molstar iframe to load this file
            js_load_code = f"""
            <script>
                (function() {{
                    document.getElementById('{iframe_id}').src = 
                        'https://molstar.org/viewer/index.html?url={file_url}';
                }})();
            </script>
            """
            
            status.value = (
                f'<span style="color: green">Loaded: {os.path.basename(file_path)}</span>'
                f'{js_load_code}'
            )
            
            # Update the served files list
            update_served_files_list()
            
        except Exception as e:
            status.value = f'<span style="color: red">Error: {str(e)}</span>'
    
    load_btn.on_click(on_load_clicked)
    
    # Remove file button
    remove_file_btn = widgets.Button(
        description='Remove Selected',
        button_style='warning',
        icon='trash',
        disabled=False
    )
    
    # Function to remove a file from the server
    def on_remove_file(b):
        if not served_files.value:
            status.value = '<span style="color: orange">Please select a file to remove</span>'
            return
            
        try:
            file_path = served_files.value
            file_server.remove_file(file_path)
            status.value = f'<span style="color: blue">Removed: {os.path.basename(file_path)}</span>'
            
            # Update the served files list
            update_served_files_list()
            
        except Exception as e:
            status.value = f'<span style="color: red">Error removing file: {str(e)}</span>'
    
    remove_file_btn.on_click(on_remove_file)
    
    # Load from served files button
    load_served_btn = widgets.Button(
        description='Load Selected',
        button_style='info',
        icon='refresh',
        disabled=False
    )
    
    # Function to load a file that's already being served
    def on_load_served(b):
        if not served_files.value:
            status.value = '<span style="color: orange">Please select a file to load</span>'
            return
            
        try:
            file_path = served_files.value
            file_url = file_server.served_files[file_path]
            
            # Update the Molstar iframe to load this file
            js_load_code = f"""
            <script>
                (function() {{
                    document.getElementById('{iframe_id}').src = 
                        'https://molstar.org/viewer/index.html?url={file_url}';
                }})();
            </script>
            """
            
            status.value = (
                f'<span style="color: green">Loaded: {os.path.basename(file_path)}</span>'
                f'{js_load_code}'
            )
            
        except Exception as e:
            status.value = f'<span style="color: red">Error loading file: {str(e)}</span>'
    
    load_served_btn.on_click(on_load_served)
    
    # Create UI layout
    file_selector = widgets.VBox([
        widgets.HTML(value='<h3>Select file from server</h3>'),
        file_type, 
        file_chooser,
        widgets.HBox([load_btn]),
        widgets.HTML(value='<h3>Currently served files</h3>'),
        served_files,
        widgets.HBox([load_served_btn, remove_file_btn]),
        status
    ])
    
    # Add a cleanup button
    cleanup_btn = widgets.Button(
        description='Stop File Server',
        button_style='danger',
        icon='trash'
    )
    
    def on_cleanup(b):
        file_server.stop()
        status.value = '<span style="color: blue">File server stopped</span>'
        update_served_files_list()
        
    cleanup_btn.on_click(on_cleanup)
    
    # Assemble the final widget
    tabs = widgets.Tab([file_selector])
    tabs.set_title(0, "Server Files")
    
    # Complete widget with cleanup option
    main_widget = widgets.VBox([
        iframe,
        tabs,
        cleanup_btn
    ])
    
    return main_widget

In [36]:
molstar_viewer = create_molstar_with_server_files(height=600, width=1200)
display(molstar_viewer)